# 문장 이진 분류 모델(다중퍼셉트론 신경망 모델)

### 0. 사용할 패키지 불러오기 

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Flatten

In [2]:
max_features = 20000   # 20,000 번째 단어까지만 사용 
text_max_words = 200   # 문장 길이는 200 단어까지만 사용 

### 1. 데이터셋 생성하기
* IMDB에서 제공하는 영화 리뷰 데이터셋을 이용
* 데이터셋은 훈련셋 25,000개, 시험셋 25,000개의 샘플을 제공
* 라벨은 1과 0으로 좋아요/싫어요로 지정
* 케라스에서 제공하는 imdb의 load_data() 함수을 이용하면 데이터셋을 쉽게 얻을 수 있음. 
* 데이터셋은 이미 정수로 인코딩되어 있으며, 정수값은 단어의 빈도수를 나타냄
* 모든 단어를 고려할 수 없으므로 빈도수가 높은 단어를 위주로 데이터셋을 생성
* 20,000번째로 많이 사용하는 단어까지만 데이터셋으로 만들고 싶다면, num_words 인자에 20000이라고 지정하면 됨.

#### 훈련셋과 시험셋 불러오기

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

* 총 25000개의 샘플
* 각 샘플은 영화 리뷰 한 건을 의미
* 단어의 인덱스로 구성되어 있습니다. 
* num_words=20000’으로 지정했기 때문에 빈도수가 20,000을 넘는 단어는 보이지가 않음
* 훈련셋 25,000개를 다시 훈련셋 20,000개와 검증셋 5,000개로 분리

#### 훈련셋과 검증셋 분리

In [4]:
x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

#### 데이터셋 전처리 : 문장 길이 맞추기
* 모델의 입력으로 사용하려면 고정된 길이로 만들어야 함
* 케라스에서 제공되는 전처리 함수인 sequence의 pad_sequences() 함수를 사용
    * 문장의 길이를 maxlen 인자로 맞춰줍니다
    * 예를 들어 200으로 지정했다면 200보다 짧은 문장은 0으로 채워서 200단어로 맞춰주고 200보다 긴 문장은 200단어까지만 잘라냅니다.
    * (num_samples, num_timesteps)으로 2차원의 numpy 배열로 만들어줍니다. 
    * maxlen을 200으로 지정하였다면, num_timesteps도 200이 됩니다.

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=text_max_words)
x_val = sequence.pad_sequences(x_val, maxlen=text_max_words)
x_test = sequence.pad_sequences(x_test, maxlen=text_max_words)

### 2. 모델 구성하기(다중퍼셉트론)
* 임베딩(Embedding) 레이어
    * 첫번째 인자(input_dim) : 단어 사전의 크기를 말하며 총 20,000개의 단어 종류가 있다는 의미입니다. 
        이 값은 앞서 imdb.load_data() 함수의 num_words 인자값과 동일해야 합니다.
    * 두번째 인자(output_dim) : 단어를 인코딩 한 후 나오는 벡터 크기 입니다. 
        이 값이 128이라면 단어를 128차원의 의미론적 기하공간에 나타낸다는 의미입니다. 
        단순하게 빈도수만으로 단어를 표시한다면, 10과 11은 빈도수는 비슷하지만 단어로 볼 때는 전혀 다른 의미를 가지고 있습니다. 
        하지만 의미론적 기하공간에서는 거리가 가까운 두 단어는 의미도 유사합니다. 
        즉 임베딩 레이어는 입력되는 단어를 의미론적으로 잘 설계된 공간에 위치시켜 벡터로 수치화 시킨다고 볼 수 있습니다.
    * input_length : 단어의 수 즉 문장의 길이를 나타냅니다. 
        임베딩 레이어의 출력 크기는 샘플 수 * output_dim * input_lenth가 됩니다. 
        임베딩 레이어 다음에 Flatten 레이어가 온다면 반드시 input_lenth를 지정해야 합니다. 
        플래튼 레이어인 경우 입력 크기가 알아야 이를 1차원으로 만들어서 Dense 레이어에 전달할 수 있기 때문입니다.

In [6]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=text_max_words))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          2560000   
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6553856   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 9,114,113
Trainable params: 9,114,113
Non-trainable params: 0
_________________________________________________________________


### 3. 모델 학습과정 설정하기

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. 모델 학습시키기

In [8]:
hist = model.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_val, y_val))

Epoch 1/2


InternalError: 2 root error(s) found.
  (0) Internal:  Blas GEMM launch failed : a.shape=(64, 25600), b.shape=(25600, 256), m=64, n=256, k=25600
	 [[node sequential/dense/MatMul (defined at <ipython-input-8-1105222e0d61>:1) ]]
	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_38]]
  (1) Internal:  Blas GEMM launch failed : a.shape=(64, 25600), b.shape=(25600, 256), m=64, n=256, k=25600
	 [[node sequential/dense/MatMul (defined at <ipython-input-8-1105222e0d61>:1) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_854]

Function call stack:
train_function -> train_function
